## ADAPTED APPLICATION TEMPLATE ERA5 - LATENT HEAT FLUXES

Package c3s_eqc_automatic_quality_control (https://github.com/bopen/c3s-eqc-automatic-quality-control).
This notebook is a first test for the implementation on the following diagnostics on the 
latent heat flux:
- Climatology Maps for the variable;
- Time Series of Globally Averaged variable;
As initial test to carry out the diagnostic proposed in the deliverable submitted on September 2022 C3S2_D520.5.1.6_202209_Consolidated_diagnostics_reanalysis_v1 
Which can be found in the +Atlantic Teams folder 
WP5 EQC of Reanalysis, Satellite and In-situ Obsv > 1_Deliverables > 09_2022 > 2_Submitted

We begin by importing the necessary modules and ignore the warnings:

In [ ]:
import warnings

import cads_toolbox
import pandas as pd
import plotly.express as px
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

warnings.filterwarnings("ignore")

#### REQUEST DEFINITION

For request definition we use **`c3s_eqc_toolbox_template.update_request_date`**.

It allows to generate the requests for a given period `[start, stop]` if stop is `None` then compute the stop month as follows:
if `current day > switch_month_day`: then `stop_month = current_month - 1`
else `stop_month = current_month - 2`

Returns the request or the list of requests for the input period.

APPLICATION SUMMARY:

- Data requests definition

- Processing:

  - chunked download
  - apply transformation to each chunk
  - cache the result on each chunk
  - merge all the chunks

- Plot the result

In [ ]:
collection_id = 'reanalysis-era5-single-levels-monthly-means'

request = {
    'variable': 'surface_latent_heat_flux',
    'year': [
        '1959', '1960', '1961',
        '1962', '1963', '1964',
        '1965', '1966', '1967',
        '1968', '1969', '1970',
        '1971', '1972', '1973',
        '1974', '1975', '1976',
        '1977', '1978', '1979',
        '1980', '1981', '1982',
        '1983', '1984', '1985',
        '1986', '1987', '1988',
        '1989', '1990', '1991',
        '1992', '1993', '1994',
        '1995', '1996', '1997',
        '1998', '1999', '2000',
        '2001', '2002', '2003',
        '2004', '2005', '2006',
        '2007', '2008', '2009',
        '2010', '2011', '2012',
        '2013', '2014', '2015',
        '2016', '2017', '2018',
        '2019', '2020', '2021',
        '2022',
    ],
    'month': [
        '01', '02', '03',
        '04', '05', '06',
        '07', '08', '09',
        '10', '11', '12',
    ],
    'time': '00:00',
    'product_type': 'monthly_averaged_reanalysis',
}
start = "1959-01"
stop = "2021-12"  # "2022-06"

Here we update the request accorfing to a start and a stop date

In [ ]:
requests = download.update_request_date(
    request, start=start, stop=stop
)
requests

We define our functions:
- an identity, to return the data just as it is (testing);
- a function which does a spatial weighted mean and resample our data into monthly mean temporal resolution;
- a function to calculate the average on each month in the dataset, i.e. a climatology function;
- a function which combines the two above to get the globally averaged climatology time series;

In [ ]:
def identity(ds: xr.Dataset) -> xr.Dataset:
    return ds
def spatial_monthly_mean(ds: xr.Dataset) -> xr.Dataset:
    ds = diagnostics.spatial_weighted_mean(ds)
    return ds.resample(time="1M").mean("time")
def monthly_climatology(ds: xr.Dataset) -> xr.Dataset:
    ds_month = ds.groupby('time.month').mean('time')
    return ds_month
def spatial_mean_climatology(ds:xr.Dataset) -> xr.Dataset:
    ds_spat_aggr = spatial_monthly_mean(ds)
    ds_spmean_monthlyclim = monthly_climatology(ds_spat_aggr)

We set up the option to store the result of our downloading and chunking into the cache locally:

In [ ]:
cads_toolbox.config.USE_CACHE = True

In [ ]:
chunks={'year':7}
monthly_slhf_maps_jm2 = download.download_and_transform(
    collection_id,
    requests,
    chunks=chunks,
    transform_func=monthly_climatology,
    concat_dim='time',
    combine='nested'
)
monthly_slhf_map_Wm2 = monthly_slhf_maps_jm2.mean('time')/86400

We have now the climatology maps.

We just need to define the spatial mean function without any resampling operation to calculate from the climatology maps the globally averaged timeseries for the climatology. 

In [ ]:
def spatial_mean(ds: xr.Dataset) -> xr.Dataset:
    ds = diagnostics.spatial_weighted_mean(ds)
    return ds
monthly_global_slhf_ts = spatial_mean(monthly_slhf_map_Wm2)

#### Plot result

In [ ]:
monthly_global_slhf_ts = monthly_global_slhf_ts.squeeze()
fig = px.line(
    x=monthly_global_slhf_ts["month"],
    y=monthly_global_slhf_ts["slhf"],
)
fig.update_layout(
    xaxis_title=r"$\mbox{Time [months]}$",
    yaxis_title=r"$\mbox{Surface Latent Heat Flux}~[W/m^2]$",
    title=r"GLOBAL MEAN MONTHLY CLIMATOLOGY SLHF 1959-2021",
)
fig.show()

In [ ]:
fig.write_image("ERA5_globalmean_monthly_climatology_slhf_1950-2021.png")

#### Now we plot the monthly climatology maps 
We define in "var" the variable to plot, and plot each month of the climatology with its contours:

In [ ]:
var = monthly_slhf_map_Wm2['slhf'].squeeze()
m = int((var.min()/10).round(2))*10
M = int((var.max()/10).round(2))*10
print(m, M)

In [ ]:
import plotly.graph_objects as go
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for i in range(len(var.month)):
    fig = go.Figure(data =
                    go.Contour(
                        x = var.longitude,
                        y = var.latitude,
                        z = var.isel(month=i).values,
                        colorscale='YlGnBu_r',
                        colorbar=dict(
                            title='W/m^2', # title here
                            titleside='right',
                            titlefont=dict(
                                size=14,
                                family='Arial, sans-serif')
                        ),
                        contours=dict(
                            start=m,
                            end=M,
                            size=30,
    )))
    fig.update_layout(
        title={
            'text': r'SLHF '+months[i]+' Climatology 1959-2021',
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis_title='longitude [degrees_east]',
        yaxis_title='latitude [degrees_north]',
    )
    fig.write_image('ERA5_climatology_map_slhf_1959-2021_'+months[i]+'.png')
    fig.show()